# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [6]:
!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import os
import gzip
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [5]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Processing...


D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Then, we define the model, object function and optimizer that we use to classify.

In [17]:

# TODO:define model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        #self.fc1构建第一个全连接层（full connect），SimpleNet.Linear是torch的全连接层方法，两个参数是左右的神经元个数
        self.fc1 = nn.Linear(28*28,500)
        self.fc2 = nn.Linear(500,10)
        
    def forward(self,x):
        x = self.fc1(x)
        #F.relu是torch的relu激活函数，还有其他激活函数，可以参考官网
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = SimpleNet()
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum = 0.9)

Next, we can start to train and evaluate!

In [33]:
# train and evaluate
loss = 0.0
correct = 0
total = 0
for epoch in range(NUM_EPOCHS):
      for i, data in enumerate(train_loader):
        (inputs, labels) =  data
        #step1，获取inputs，计算outputs
        #因为是全连接网络，所以需要reshape到一维，卷积不需要
        inputs = inputs.reshape(-1, 28*28)
        outputs = model(inputs)
        _, predict = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predict == labels).sum()
        #step2，清零梯度，计算loss，反向传播
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i%100 == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'
                      .format(epoch + 1, NUM_EPOCHS, loss.item())) 

Epoch [1/10], Loss: 0.0031
Epoch [1/10], Loss: 0.0016
Epoch [1/10], Loss: 0.0050
Epoch [1/10], Loss: 0.0031
Epoch [1/10], Loss: 0.0037
Epoch [2/10], Loss: 0.0014
Epoch [2/10], Loss: 0.0011
Epoch [2/10], Loss: 0.0014
Epoch [2/10], Loss: 0.0019
Epoch [2/10], Loss: 0.0028
Epoch [3/10], Loss: 0.0021
Epoch [3/10], Loss: 0.0026
Epoch [3/10], Loss: 0.0033
Epoch [3/10], Loss: 0.0024
Epoch [3/10], Loss: 0.0028
Epoch [4/10], Loss: 0.0027
Epoch [4/10], Loss: 0.0015
Epoch [4/10], Loss: 0.0046
Epoch [4/10], Loss: 0.0013
Epoch [4/10], Loss: 0.0005
Epoch [5/10], Loss: 0.0028
Epoch [5/10], Loss: 0.0025
Epoch [5/10], Loss: 0.0009
Epoch [5/10], Loss: 0.0026
Epoch [5/10], Loss: 0.0022
Epoch [6/10], Loss: 0.0030
Epoch [6/10], Loss: 0.0016
Epoch [6/10], Loss: 0.0011
Epoch [6/10], Loss: 0.0026
Epoch [6/10], Loss: 0.0020
Epoch [7/10], Loss: 0.0022
Epoch [7/10], Loss: 0.0013
Epoch [7/10], Loss: 0.0021
Epoch [7/10], Loss: 0.0026
Epoch [7/10], Loss: 0.0024
Epoch [8/10], Loss: 0.0017
Epoch [8/10], Loss: 0.0022
E

#### Q5:
Please print the training and testing accuracy.

In [43]:
def test(test_loader,model):
    correct = 0
    total = 0
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.reshape(-1, 28*28)
        outputs = model(inputs)
        #torch.max并不是np.max一个意思，是用以计算sofamax的分类类别的，建议CSDN查一下
        _, predict = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predict == labels).sum()
    print('Epoch {} testing accuracy:{}'.format(epoch+1, 100*correct/total))

print('Epoch {} training accuracy:{}'.format(epoch+1, 100*correct/total))
test(test_loader,model)
    

Epoch 10 training accuracy:99.99883270263672
Epoch 10 testing accuracy:98.28726196289062
